In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, RBF, Matern, RationalQuadratic
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

In [8]:
from tqdm import tqdm
import time

# Data Load 

In [17]:
def data_loading():
    """
    This function loads the training and test data, preprocesses it, removes the NaN values and interpolates the missing 
    data using imputation

    Parameters
    ----------
    Returns
    ----------
    X_train: matrix of floats, training input with features
    y_train: array of floats, training output with labels
    X_test: matrix of floats: dim = (100, ?), test input with features
    """
    # Load training data
    train_df = pd.read_csv("train.csv")
    
    print("Training data:")
    print("Shape:", train_df.shape)
    print(train_df.head(2))
    print('\n')
    
    # Load test data
    test_df = pd.read_csv("test.csv")

    print("Test data:")
    print(test_df.shape)
    print(test_df.head(2))

    # Dummy initialization of the X_train, X_test and y_train   
    X_train = np.zeros_like(train_df.drop(['price_CHF'],axis=1))
    y_train = np.zeros_like(train_df['price_CHF'])
    X_test = np.zeros_like(test_df)

    # TODO: Perform data preprocessing, imputation and extract X_train, y_train and X_test
    train_arr = np.array(train_df)
    test_arr = np.array(test_df)
    
    # one hot encoder
    train_arr = np.array(train_df)
    test_arr = np.array(test_df)
    
    # one hot encoder
    column_to_encode = 0
    train_to_encode = train_arr[:, column_to_encode].reshape(-1, 1)
    test_to_encode = test_arr[:, column_to_encode].reshape(-1, 1)
    encoder = OneHotEncoder()
    encoder.fit(train_to_encode)
    encoded_train_data = encoder.transform(train_to_encode)
    encoded_train_arr = np.concatenate((train_arr[:, :column_to_encode],
                            encoded_train_data.toarray(),
                            train_arr[:, column_to_encode+1:]), axis=1)
    
    encoded_test_data = encoder.transform(test_to_encode)
    encoded_test_arr = np.concatenate((test_arr[:, :column_to_encode],
                            encoded_test_data.toarray(),
                            test_arr[:, column_to_encode+1:]),axis=1)
    
    # KNN Imputer
    imputer = KNNImputer(n_neighbors=5, weights = 'distance')
    imputed_train = imputer.fit_transform(encoded_train_arr)
    imputed_test = imputer.fit_transform(encoded_test_arr)
    
    # delete the price of electric of swi
    train_idx = [i for i in range(encoded_train_arr.shape[0]) if np.isnan(encoded_train_arr[i,5]) == False] 
    imputed_train_refined = imputed_train[train_idx]
    
    X_train = np.delete(imputed_train_refined, 5, 1)
    y_train = imputed_train_refined[:, 5]
    
    X_test = imputed_test
    
    assert (X_train.shape[1] == X_test.shape[1]) and (X_train.shape[0] == y_train.shape[0]) and (X_test.shape[0] == 100), "Invalid data shape"
    return X_train, y_train, X_test

In [18]:
X_train, y_train, X_test = data_loading()

Training data:
Shape: (900, 11)
   season  price_AUS  price_CHF  price_CZE  price_GER  price_ESP  price_FRA  \
0  spring        NaN   9.644028  -1.686248  -1.748076  -3.666005        NaN   
1  summer        NaN   7.246061  -2.132377  -2.054363  -3.295697  -4.104759   

   price_UK  price_ITA  price_POL  price_SVK  
0 -1.822720  -3.931031        NaN  -3.238197  
1 -1.826021        NaN        NaN  -3.212894  


Test data:
(100, 10)
   season  price_AUS  price_CZE  price_GER  price_ESP  price_FRA  price_UK  \
0  spring        NaN   0.472985   0.707957        NaN  -1.136441 -0.596703   
1  summer  -1.184837   0.358019        NaN  -3.199028  -1.069695       NaN   

   price_ITA  price_POL  price_SVK  
0        NaN   3.298693   1.921886  
1  -1.420091   3.238307        NaN  


# Model Test and Optimize

## Kernel Optimization 

### Matern Kernel 

In [ ]:
ls_range = np.logspace(-5, 1, 100, base=10)
nu_range = np.logspace(-5, 1, 100, base=10)

In [ ]:
params = [  (0.05, 0.5), (0.05, 1.5), (0.05, 2.5),
                (0.1, 0.5), (0.1, 1.5), (0.1, 2.5),
                (1, 0.5), (1, 1.5), (1, 2.5)]
for i in range (len(ls_range)):
    for j in range (len(nu_range)):
        params.append((ls_range[i], nu_range[j]))

In [28]:
def Matern_para_select(X, y, n_folds):
    print("————Matern————")
    ls_range = np.logspace(-2, 1, 10, base=10)
    nu_range = np.logspace(-2, 1, 10, base=10)
    params = [  (0.05, 0.5), (0.05, 1.5), (0.05, 2.5),
                (0.1, 0.5), (0.1, 1.5), (0.1, 2.5),
                (1, 0.5), (1, 1.5), (1, 2.5)]
#     for i in range (len(ls_range)):
#         for j in range (len(nu_range)):
#             params.append((ls_range[i], nu_range[j]))
    R2score_mat = np.zeros((n_folds, len(params)))
    kf = KFold(n_splits=n_folds, shuffle=True, random_state= 14)
    for fold_idx, (train, test) in enumerate(kf.split(X)):
        X_train, X_val, y_train, y_val = X[train], X[test], y[train], y[test]
        for idx, (ls, nu) in enumerate(params):
            gpr = GaussianProcessRegressor(kernel= Matern(length_scale=ls, nu=nu))
            gpr.fit(X_train, y_train)
            y_val_pred = gpr.predict(X_val)
            R2score_mat[fold_idx][idx] = r2_score(y_val, y_val_pred)

    avg_R2score = np.mean(R2score_mat, axis=0)
    print("Best Param: ", params[np.argmax([avg_R2score])], " Score: ", avg_R2score[np.argmax([avg_R2score])])
    # print(avg_R2score)
    print("______________end_____________")
    return avg_R2score, params[np.argmax([avg_R2score])]

In [29]:
n_folds = 10
avg_R2score_M, best_para_M = Matern_para_select(X_train, y_train, n_folds)

————Matern————
Best Param:  (1, 0.5)  Score:  0.9831471878381295
______________end_____________


In [56]:
gpr = GaussianProcessRegressor(kernel=Matern(length_scale=1, nu=0.5))
gpr.fit(X_train, y_train)
y_pred = gpr.predict(X_test)

In [57]:
# Save results in the required format
dt = pd.DataFrame(y_pred) 
dt.columns = ['price_CHF']
dt.to_csv('results.csv', index=False)

### Rational Quadratic Kernel 

In [24]:
def RationalQuardratic_para_select(X, y, n_folds):
    print("————Rational Quadratic————")
    ls_range = np.logspace(-2, 0.5, 10, base=10)
    alp_range = np.logspace(-2, 0.5, 10, base=10)
    params = [  (0.1, 0.1), (0.1, 1),
                (1, 0.1), (1, 1)]
#     for i in range(len(ls_range)):
#         for j in range(len(alp_range)):
#             params.append((ls_range[i], alp_range[j]))
    R2score_mat = np.zeros((n_folds, len(params)))    
    kf = KFold(n_splits=n_folds, shuffle=True, random_state= 14)
    for fold_idx, (train, test) in enumerate(kf.split(X)):
        X_train, X_val, y_train, y_val = X[train], X[test], y[train], y[test]
        for idx, (ls, alp) in enumerate(params):
            gpr = GaussianProcessRegressor(kernel = RationalQuadratic(length_scale=ls, alpha=alp))
            gpr.fit(X_train, y_train)
            y_val_pred = gpr.predict(X_val)
            R2score_mat[fold_idx][idx] = r2_score(y_val, y_val_pred)
            
    avg_R2score = np.mean(R2score_mat, axis=0)
    print("Best Param: ", params[np.argmax([avg_R2score])], " Score: ", avg_R2score[np.argmax([avg_R2score])])
    # print(avg_R2score)
    print("______________end_____________")
    return avg_R2score, params[np.argmax([avg_R2score])]

In [25]:
n_folds = 10
avg_R2score, best_para_RQ = RationalQuardratic_para_select(X_train, y_train, n_folds)

————Rational Quadratic————
Best Param:  (0.1, 1)  Score:  0.9806467592204626
______________end_____________


### Mix Kernel

###  Rational Quadratic + Matern

In [34]:
def RQM_sup_para_select(X, y, n_folds):
    print("————Rational Quadratic + Matern————")
    RQ_params = [  (0.1, 0.1), (0.1, 0.5),(0.1, 1), 
                 (0.5, 0.1), (0.5, 0.5), (0.5, 1),
                (1, 0.1), (1, 0.5), (1, 1)]
    M_params = [ (0.05, 0.2), (0.05, 0.4), (0.05, 0.5), (0.05, 1.5), (0.05, 2.5),
                (0.1, 0.2), (0.1, 0.4),(0.1, 0.5), (0.1, 1.5), (0.1, 2.5),
                (0.25, 0.2), (0.25, 0.4),(0.25, 0.5), (0.25, 1.5), (0.25, 2.5),
                (0.4, 0.2), (0.4, 0.4),(0.4, 0.5), (0.4, 1.5), (0.4, 2.5),
                (0.6, 0.2), (0.6, 0.4),(0.6, 0.5), (0.6, 1.5), (0.6, 2.5),
                (1, 0.2), (1, 0.4),(1, 0.5), (1, 1.5), (1, 2.5)]

    params = []
    for i in range(len(RQ_params)):
        for j in range(len(M_params)):
            params.append((RQ_params[i], M_params[j]))
    params = []
    for i in range(len(RQ_params)):
        for j in range(len(M_params)):
            params.append((RQ_params[i], M_params[j]))
    R2score_mat = np.zeros((n_folds, len(params)))    
    kf = KFold(n_splits=n_folds, shuffle=True, random_state= 14)
    pbar = tqdm(total=len(params)*n_folds)
    for fold_idx, (train, test) in enumerate(kf.split(X)):
        X_train, X_val, y_train, y_val = X[train], X[test], y[train], y[test]
        for idx, (RQ, M) in enumerate(params):
            kernel = RationalQuadratic(length_scale=RQ[0], alpha=RQ[1]) + Matern(length_scale=M[0], nu=M[1])
            gpr = GaussianProcessRegressor(kernel = kernel, n_restarts_optimizer=9)
            gpr.fit(X_train, y_train)
            y_val_pred = gpr.predict(X_val)
            R2score_mat[fold_idx][idx] = r2_score(y_val, y_val_pred)
            pbar.update()
    pbar.close()            
    avg_R2score = np.mean(R2score_mat, axis=0)
    print("Best Param: ", params[np.argmax([avg_R2score])], " Score: ", avg_R2score[np.argmax([avg_R2score])])
    # print(avg_R2score)
    print("______________end_____________")
    return avg_R2score, params[np.argmax([avg_R2score])]       

In [35]:
n_folds = 10
avg_R2score_sup, best_para_sup = RQM_sup_para_select(X_train, y_train, n_folds)

  0%|                                                  | 0/2700 [00:00<?, ?it/s]

————Rational Quadratic + Matern————


/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
  0%|                                       | 1/2700 [00:35<26:50:12, 35.80s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
  0%|                                       | 2/2700 [01:18<28:23:43, 37.89s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the 

  1%|▍                                     | 32/2700 [08:57<12:14:12, 16.51s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
  1%|▌                                      | 35/2700 [09:12<6:35:22,  8.90s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
  1%|▌                                     | 36/2700 [09:44<11:42:21, 15.82s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning:

  2%|▉                                     | 67/2700 [16:48<11:09:48, 15.26s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
  3%|█                                      | 70/2700 [17:00<5:40:56,  7.78s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
  3%|█                                      | 71/2700 [17:22<8:50:05, 12.10s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning:

...

 95%|██████████████████████████████████▎ | 2575/2700 [11:21:30<21:58, 10.55s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
 95%|██████████████████████████████████▎ | 2576/2700 [11:22:08<38:33, 18.66s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
 95%|██████████████████████████████████▎ | 2577/2700 [11:22:48<51:14, 25.00s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning:

 97%|██████████████████████████████████▊ | 2610/2700 [11:30:19<12:56,  8.63s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
 97%|██████████████████████████████████▊ | 2611/2700 [11:30:43<19:48, 13.36s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
 97%|██████████████████████████████████▊ | 2612/2700 [11:31:02<21:54, 14.93s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning:

 98%|███████████████████████████████████▎| 2645/2700 [11:39:29<09:42, 10.59s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
 98%|███████████████████████████████████▎| 2646/2700 [11:39:57<14:11, 15.77s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
 98%|███████████████████████████████████▎| 2650/2700 [11:40:45<08:20, 10.00s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning:

 99%|███████████████████████████████████▊| 2685/2700 [11:49:58<02:43, 10.87s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
 99%|███████████████████████████████████▊| 2686/2700 [11:50:30<04:03, 17.42s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
100%|███████████████████████████████████▊| 2690/2700 [11:51:12<01:46, 10.68s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning:

Best Param:  ((0.1, 0.1), (0.05, 0.2))  Score:  0.9863758493123134
______________end_____________


In [42]:
kernel= RationalQuadratic(length_scale=0.1, alpha=0.1) + Matern(length_scale=0.05, nu=0.2)
gpr = GaussianProcessRegressor(kernel = kernel)
gpr.fit(X_train, y_train)
y_pred = gpr.predict(X_test) 

/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


In [43]:
# Save results in the required format
dt = pd.DataFrame(y_pred) 
dt.columns = ['price_CHF']
dt.to_csv('results.csv', index=False)

### Rational Quadratic * Matern

In [36]:
def RQM_mul_para_select(X, y, n_folds):
    print("————Rational Quadratic * Matern————")
    RQ_params = [  (0.1, 0.1), (0.1, 0.5),(0.1, 1), 
                 (0.5, 0.1), (0.5, 0.5), (0.5, 1),
                (1, 0.1), (1, 0.5), (1, 1)]
    M_params = [ (0.05, 0.2), (0.05, 0.4), (0.05, 0.5), (0.05, 1.5), (0.05, 2.5),
                (0.1, 0.2), (0.1, 0.4),(0.1, 0.5), (0.1, 1.5), (0.1, 2.5),
                (0.25, 0.2), (0.25, 0.4),(0.25, 0.5), (0.25, 1.5), (0.25, 2.5),
                (0.4, 0.2), (0.4, 0.4),(0.4, 0.5), (0.4, 1.5), (0.4, 2.5),
                (0.6, 0.2), (0.6, 0.4),(0.6, 0.5), (0.6, 1.5), (0.6, 2.5),
                (1, 0.2), (1, 0.4),(1, 0.5), (1, 1.5), (1, 2.5)]

    params = []
    for i in range(len(RQ_params)):
        for j in range(len(M_params)):
            params.append((RQ_params[i], M_params[j]))
    R2score_mat = np.zeros((n_folds, len(params)))    
    kf = KFold(n_splits=n_folds, shuffle=True, random_state= 14)
    pbar = tqdm(total=len(params)*n_folds)
    for fold_idx, (train, test) in enumerate(kf.split(X)):
        X_train, X_val, y_train, y_val = X[train], X[test], y[train], y[test]
        for idx, (RQ, M) in enumerate(params):
            kernel = RationalQuadratic(length_scale=RQ[0], alpha=RQ[1]) * Matern(length_scale=M[0], nu=M[1])
            gpr = GaussianProcessRegressor(kernel = kernel, n_restarts_optimizer=9)
            gpr.fit(X_train, y_train)
            y_val_pred = gpr.predict(X_val)
            R2score_mat[fold_idx][idx] = r2_score(y_val, y_val_pred)
            pbar.update()
    pbar.close()            
    avg_R2score = np.mean(R2score_mat, axis=0)
    print("Best Param: ", params[np.argmax([avg_R2score])], " Score: ", avg_R2score[np.argmax([avg_R2score])])
    # print(avg_R2score)
    print("______________end_____________")
    return avg_R2score, params[np.argmax([avg_R2score])]       

In [37]:
n_folds = 10
avg_R2score_mul, best_para_mul = RQM_mul_para_select(X_train, y_train, n_folds)

  0%|                                                  | 0/2700 [00:00<?, ?it/s]

————Rational Quadratic * Matern————


  0%|                                       | 2/2700 [00:49<20:42:31, 27.63s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
  0%|                                       | 3/2700 [00:53<15:33:25, 20.77s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
  0%|                                       | 7/2700 [01:47<13:48:41, 18.46s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: Convergence

  2%|▊                                     | 57/2700 [12:27<12:44:11, 17.35s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
  2%|▊                                      | 58/2700 [12:31<9:40:28, 13.18s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
  2%|▊                                     | 62/2700 [13:39<14:09:18, 19.32s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: Convergence

  4%|█▌                                   | 112/2700 [24:52<12:02:13, 16.74s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
  4%|█▌                                    | 113/2700 [24:57<9:33:39, 13.30s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
  4%|█▌                                   | 117/2700 [26:16<16:41:42, 23.27s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: Convergence

...

 24%|████████▊                           | 658/2700 [2:26:58<9:33:43, 16.86s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
 24%|████████▊                           | 659/2700 [2:27:00<6:58:12, 12.29s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
 25%|████████▌                          | 662/2700 [2:28:04<10:36:25, 18.74s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: Convergence

 26%|█████████▎                          | 694/2700 [2:35:14<5:36:38, 10.07s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
 26%|█████████▎                          | 697/2700 [2:36:03<8:34:30, 15.41s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
 26%|█████████▎                          | 699/2700 [2:36:08<4:49:18,  8.67s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: Convergence

 27%|█████████▊                          | 738/2700 [2:44:58<8:11:06, 15.02s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
 27%|█████████▊                          | 739/2700 [2:45:00<6:01:39, 11.07s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
 27%|█████████▉                          | 742/2700 [2:45:38<7:05:03, 13.03s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: Convergence

 29%|██████████▎                         | 778/2700 [2:53:46<7:39:46, 14.35s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
 29%|██████████▍                         | 779/2700 [2:53:49<5:50:48, 10.96s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
 29%|██████████▍                         | 780/2700 [2:53:53<4:46:05,  8.94s/it]/home/mullin/anaconda3/envs/PAIML/lib/python3.9/site-packages/sklearn/gaussian_process/_gpr.py:629: ConvergenceWar

KeyboardInterrupt: 

In [28]:
kernel= RationalQuadratic(length_scale=0.1, alpha=1) * Matern(length_scale=1, nu=2.5)
gpr = GaussianProcessRegressor(kernel=kernel)
gpr.fit(X_train, y_train)
y_pred = gpr.predict(X_test)

In [29]:
# Save results in the required format
dt = pd.DataFrame(y_pred) 
dt.columns = ['price_CHF']
dt.to_csv('results.csv', index=False)